<a href="https://colab.research.google.com/github/patzaa/Entity_Recognition_BERT/blob/master/NER_BERT_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [12]:
pip install transformers

Mounting the dataset 


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Exploring the dataset

In [19]:
import pandas as pd

df = pd.read_csv("../content/drive/MyDrive/NER_Dataset/ner_dataset.csv", index_col=0 , encoding ='latin1') 
df.head()

,Word,POS,Tag
Sentence #,,,
Sentence: 1,Thousands,NNS,O
NaN,of,IN,O
NaN,demonstrators,NNS,O
NaN,have,VBP,O
NaN,marched,VBN,O


 # Dataset class


In [20]:
import transformers
import torch 

MAX_LEN = 128
LR = 1e-5
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 10
BASE_MODEL_PATH = "../bert_model"
MODEL_PATH = "model.bin"
TRAINING_FILE = "../dataset/ner_dataset.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(
    BASE_MODEL_PATH,
    do_lower_case=True
)

HTTPError: ignored

In [21]:
from transformers import BertTokenizer

class NERDataset:
    def __init__(self, texts, pos, tags):
        # texts: [["hey", "I", "am", "Dan"], ["hello".....]]
        # pos/tags: [[1 2 3 4 1 5], [....].....]]
        self.texts = texts
        self.pos = pos
        self.tags = tags
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        pos = self.pos[idx]
        tags = self.tags[idx]
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

        ids = []
        target_pos = []
        target_tag =[]

        for i, s in enumerate(text):

            inputs = tokenizer.encode(
                s,
                add_special_tokens=False
            )
            
            input_len = len(inputs)
            ids.extend(inputs)
            target_pos.extend([pos[i]] * input_len)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_pos = target_pos[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [101] + ids + [102]
        target_pos = [0] + target_pos + [0]
        target_tag = [0] + target_tag + [0]

        mask = [1] * len(ids)
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)
        
        ids = ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_pos = target_pos + ([0] * padding_len)
        target_tag = target_tag + ([0] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_pos": torch.tensor(target_pos, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }


# Loss function and model class 

In [22]:
import torch
from transformers import BertModel
import torch.nn as nn

def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss


class BERTModel(nn.Module):
    def __init__(self, num_tag, num_pos):
        super(BERTModel, self).__init__()
        self.num_tag = num_tag
        self.num_pos = num_pos
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.bert_drop_1 = nn.Dropout(0.3)
        self.bert_drop_2 = nn.Dropout(0.3)
        self.out_tag = nn.Linear(768, self.num_tag)
        self.out_pos = nn.Linear(768, self.num_pos)
    
    def forward(self, ids, mask, token_type_ids, target_pos, target_tag):
        o1, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        
         
        bo_tag = self.bert_drop_1(o1)
        bo_pos = self.bert_drop_2(o1)

        tag = self.out_tag(bo_tag)
        pos = self.out_pos(bo_pos)

        loss_tag = loss_fn(tag, target_tag, mask, self.num_tag)
        loss_pos = loss_fn(pos, target_pos, mask, self.num_pos)

        loss = (loss_tag + loss_pos) / 2

        return tag, pos, loss


In [ ]:
import pandas as pd
import numpy as np

import joblib
import torch

from sklearn import preprocessing
from sklearn import model_selection

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from tqdm import tqdm


def process_data(data_path):
    df = pd.read_csv(data_path, encoding="latin-1")
    df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")

    enc_pos = preprocessing.LabelEncoder()
    enc_tag = preprocessing.LabelEncoder()

    df.loc[:, "POS"] = enc_pos.fit_transform(df["POS"])
    df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])

    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    pos = df.groupby("Sentence #")["POS"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, pos, tag, enc_pos, enc_tag

def train_fn(data_loader, model, optimizer, device, scheduler):
  model.train()
  final_loss = 0
  for data in tqdm(data_loader, total=len(data_loader)):
    for k, v in data.items():
      data[k] = v.to(device)
      optimizer.zero_grad()
      _, _, loss = model(**data)
      loss.backward()
      optimizer.step()
      scheduler.step()
      final_loss += loss.item()
  return final_loss / len(data_loader)

def eval_fn(data_loader, model, device):
  model.eval()
  final_loss = 0
  for data in tqdm(data_loader, total=len(data_loader)):
    for k, v in data.items():
      data[k] = v.to(device)
      _, _, loss = model(**data)
      final_loss += loss.item()
  return final_loss / len(data_loader)

if __name__ == "__main__":
    sentences, pos, tag, enc_pos, enc_tag = process_data("../content/drive/MyDrive/NER_Dataset/ner_dataset.csv")
    
    meta_data = {
        "enc_pos": enc_pos,
        "enc_tag": enc_tag
    }

    joblib.dump(meta_data, "meta.bin")

    num_pos = len(list(enc_pos.classes_))
    num_tag = len(list(enc_tag.classes_))

    (
        train_sentences,
        test_sentences,
        train_pos,
        test_pos,
        train_tag,
        test_tag
    ) = model_selection.train_test_split(sentences, pos, tag, random_state=42, test_size=0.1)

    train_dataset = NERDataset(
        texts=train_sentences, pos=train_pos, tags=train_tag
    )





    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=2
    )

    valid_dataset = NERDataset(
        texts=test_sentences, pos=test_pos, tags=test_tag
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
    )

    
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = BERTModel(num_tag=num_tag, num_pos=num_pos)
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=LR)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
    )

    best_loss = np.inf

 

    for epoch in range(EPOCHS):
        train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
        test_loss = eval_fn(valid_data_loader, model, device)
        print(f"Train Loss = {train_loss} Valid Loss = {test_loss}")
        if test_loss < best_loss:
            torch.save(model.state_dict(), MODEL_PATH)
            best_loss = test_loss








  0%|          | 0/1349 [00:00<?, ?it/s]

  1%|          | 11/1349 [37:39<73:59:40, 199.09s/it]